# 数据拆分流程

In [1]:
import pandas as pd
import numpy as np
import sys,glob,os,warnings
import yaml

warnings.filterwarnings("ignore")
sys.path.append("/home/zhangli_lab/zhouxiangyu/DATA/workflow/split-mgi-lanefastq/rules")

from mgi_scripts import *

# 1. 整理index 文件




In [2]:
config='''
cycletype: r100e1r100e1b8b8

ChipID: V350264101


lane_range: "L01,L02,L03,L04"

step: 1

index_file: Sample_index.csv

rawfq_dir : /home/zhangli_lab/zhouxiangyu/DATA/mgi2000/rawfq/

temp_dir: temp_dir/ 

outfq_dir: outfq_dir/
'''
yamlfile=os.path.join("config.yaml")
with open(yamlfile,'w') as f:
    f.write(config)
    
with open(yamlfile) as f:
    info = yaml.load(f, Loader=yaml.FullLoader)


In [3]:
excel=get_standard_index("%s.index.csv"%info["ChipID"])
excel.libtype=excel.libtype.replace("Smartseq",'Smart-seq')
excel.libtype=excel.libtype.replace("unkown",'Smart-seq')
excel.sampleName=excel.sampleName.str.replace("/",'_').str.replace(" ",'-')
excel.batch=excel.batch.str.replace("/",'_').str.replace(" ",'-')
excel.belonguser=excel.belonguser.replace("test","戴睿成")


#excel.libtype=excel.libtype.replace("","unkown")
#excel.libtype=excel.apply(lambda x:"真核普通转录组" if x.belonguser=="吴为" else x.libtype,axis=1)
#excel.libtype=excel.apply(lambda x:"Smart-seq" if x.libtype=="ATAC" else x.libtype,axis=1)

excel[['Lane','Lab','libtype']].value_counts().sort_index()


Lane  Lab     libtype  
L01   罗敏敏实验室  Smart-seq     384
              真核链特异性          4
      陈坚实验室   Smart-seq      68
L02   井淼实验室   DNA常规小片段        2
      罗敏敏实验室  Smart-seq    1152
              真核链特异性          4
L03   罗敏敏实验室  Smart-seq    1920
L04   罗敏敏实验室  Smart-seq    1920
Name: count, dtype: int64

In [4]:
df1=excel[excel.libtype=="Smart-seq"]
df2=excel[excel.libtype!="Smart-seq"]
SampleInfo=pd.concat([smartseq_index(df1),df2])
SampleInfo.Lab=SampleInfo.Lab.apply(info_zh_to_en)
SampleInfo.belonguser=SampleInfo.belonguser.apply(info_zh_to_en)
SampleInfo.to_csv("Sample_index.csv",index=False)
SampleInfo[['Lane','Lab',"belonguser",'batch','libtype']].value_counts().to_frame("Sample_num").sort_index().reset_index()

,Lane,Lab,belonguser,batch,libtype,Sample_num
0,L01,ChenLab,FuJY,LPS-neuron,Smartseq,68
1,L01,LuoLab,DaiRC,DRC-cOpn5-day1-1-index20,Smartseq,384
2,L01,LuoLab,WangYi,WY-1,真核链特异性,4
3,L02,JingLab,WangYa,0-1_10-2,DNA常规小片段,2
4,L02,LuoLab,DaiRC,DRC-cOpn5-day1-2-index1,Smartseq,384
5,L02,LuoLab,DaiRC,DRC-cOpn5-day1-7-index21,Smartseq,384
6,L02,LuoLab,DaiRC,DRC-cOpn5-day1-9-index22,Smartseq,384
7,L02,LuoLab,WangYi,WY-2,真核链特异性,4
8,L03,LuoLab,DaiRC,DRC-Cx3crl-cOpn5-3w-1-index4,Smartseq,384
9,L03,LuoLab,DaiRC,DRC-Cx3crl-cOpn5-3w-2-index5,Smartseq,384


#### 确认是否存在命名重复的问题

In [6]:
SampleInfo.sampleName.value_counts().value_counts()

count
1    5454
Name: count, dtype: int64

In [7]:
SampleInfo[SampleInfo.sampleName.duplicated(keep=False)]

,sampleName,i5,i7,CHIP,Lane,Lab,belonguser,batch,libtype


#### 确认是否存在index重复的问题

In [8]:
remove_list=SampleInfo[SampleInfo[["Lane",'i5','i7']].duplicated(keep=False)]
remove_list

,sampleName,i5,i7,CHIP,Lane,Lab,belonguser,batch,libtype


#### 去除重复index 样品

In [17]:
SampleInfo[SampleInfo.sampleName.isin(remove_list.sampleName)==False].to_csv("Sample_index.csv",index=False)

# 运行拆分程序

```sh
conda activate snakePipes

srun -p q_fat_z 
snakemake -j10 --configfile config.yaml --config step=1 \
--snakefile /home/zhangli_lab/zhouxiangyu/DATA/workflow/mgi2000/Snakefile \
--cluster "sbatch -N 1 -c 6 -p q_fat,q_cn,q_fat_z --mem 50G" -np


snakemake -j10 --configfile config.yaml --config step=2 \
--snakefile /home/zhangli_lab/zhouxiangyu/DATA/workflow/mgi2000/Snakefile \
--cluster "sbatch -N 1 -c 6 -p q_fat,q_cn,q_fat_z --mem 50G" -np
```

# 检查未拆分序列

In [6]:
for lane in ["L01","L02","L03","L04"]:
    print(lane)
    check_lane(lane,SampleInfo)
    print("\n")

L01
I5_Drc I7_Drc
Lane  belonguser  batch                   
L02   DaiRC       DRC-cOpn5-day1-9-index22    28
Name: count, dtype: int64


L02


L03


L04




# 统计

In [7]:
import pandas as pd
import glob

In [9]:
pd.read_csv("temp_dir/%s.stat.txt"%info["ChipID"],sep='\t')


,Chip,Lane,ChipProductivity(%),CycleNumber,ImageArea,Q30(%),SplitRate(%),TotalReads(M)
0,V350264101,L01,72.85,216,432,94.71,94.39,418.28
1,V350264101,L02,76.77,216,432,94.34,96.15,449.00
2,V350264101,L03,77.21,216,432,95.05,96.86,454.92
3,V350264101,L04,75.90,216,432,96.56,97.10,448.29


In [10]:
outdict={}
for i in SampleInfo.Lab.unique()[1:]:
    res=get_stat("./outfq_dir/%s/"%i)
    res=res.merge(SampleInfo[["sampleName",'libtype']],on="sampleName")
    res.batch=res.apply(lambda x:"-".join(x.sampleName.split("-")[:-1]) if x.libtype=="Smartseq" else x.batch,axis=1)
    out=get_stat_result(res,["user",'batch']).sort_index(ascending=False)
    outdict[i]=out
    print(out)

                 Samples Size(G) reads(M)
user batch                               
FuJY LPS-neuron       68   31.59   157.97
                 Samples Size(G) reads(M)
user   batch                             
WangYa 0-1_10-2        2    0.54     2.68


In [13]:
outdict["ChenLab"]

,,Samples,Size(G),reads(M)
user,batch,,,
FuJY,LPS-neuron,68,31.59,157.97


In [14]:
outdict["JingLab"]

,,Samples,Size(G),reads(M)
user,batch,,,
WangYa,0-1_10-2,2,0.54,2.68


In [16]:
i="JingLab"
res=get_stat("./outfq_dir/%s/"%i)
res

,sampleName,batch,ReadNum,ReadLength,GC%,Q10%,Q20%,Q30%,BaseNum,user
0,0-1,0-1_10-2,1330123,100,52.17,99.34,92.52,81.95,266024600,WangYa
1,10-2,0-1_10-2,1349134,100,52.11,99.31,92.48,81.88,269826800,WangYa
